In [1]:
!pip install transformers datasets
!sudo apt-get install git-lfs


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 4.1 MB/s 
     |████████████████████████████████| 441 kB 71.6 MB/s 
     |████████████████████████████████| 163 kB 91.8 MB/s 
     |████████████████████████████████| 7.6 MB 56.5 MB/s 
     |████████████████████████████████| 212 kB 61.0 MB/s 
     |████████████████████████████████| 115 kB 93.9 MB/s 
     |████████████████████████████████| 127 kB 90.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 22 no

In [2]:
from huggingface_hub import notebook_login

notebook_login()


Login successful
Your token has been saved to /root/.huggingface/token


In [3]:
from huggingface_hub import HfApi

user_id = HfApi().whoami()["name"]

print(f"user id '{user_id}' will be used during the example")

user id 'ellabettison' will be used during the example


In [13]:
import csv
jsonl = []
with open('/content/drive/MyDrive/org_name_parsing/company_names_all.csv', newline='') as csvfile:
     spamreader = csv.reader(csvfile, quoting=csv.QUOTE_NONE)
     for row in spamreader:
       new_row = row[0].replace('"', '')
       new_row = new_row.replace("'", "")
       new_row = new_row.replace("\\", "")
       if new_row != "":
        jsonl.append(new_row)
        # jsonl.append({"text":new_row})
       else:
        print("found it!")

jsonl = jsonl[:1_000_000]
print(jsonl[-1])

found it!
planetworks consulting


In [ ]:
from tqdm import tqdm
from transformers import BertTokenizerFast

# repositor id for saving the tokenizer
tokenizer_id="finetuned_orgnames_roberta"

# create a python generator to dynamically load the data
def batch_iterator(batch_size=10000):
    for i in tqdm(range(0, len(jsonl), batch_size)):
        yield jsonl[i : i + batch_size] #["text"]

# create a tokenizer from existing one to re-use special tokens
tokenizer = BertTokenizerFast.from_pretrained("roberta-base", model_max_length=10)

In [ ]:
bert_tokenizer = tokenizer.train_new_from_iterator(text_iterator=batch_iterator(), vocab_size=32_000)
bert_tokenizer.save_pretrained("tokenizer")

In [ ]:
bert_tokenizer.push_to_hub(tokenizer_id)

In [14]:
from transformers import AutoTokenizer
import multiprocessing
import pandas as pd
from datasets import Dataset

# load tokenizer
#tokenizer = AutoTokenizer.from_pretrained(f"{user_id}/{tokenizer_id}", model_max_length=10)
tokenizer = AutoTokenizer.from_pretrained("roberta-base", model_max_length=16)
num_proc = multiprocessing.cpu_count()
print(f"The max length for the tokenizer is: {tokenizer.model_max_length}")

examples_df = pd.DataFrame(jsonl, columns=['text'])
print(examples_df)
dataset = Dataset.from_pandas(examples_df)

print(dataset)

def group_texts(examples):
    tokenized_inputs = tokenizer(
       examples['text'], return_special_tokens_mask=True, truncation=True, 
       max_length=tokenizer.model_max_length, return_attention_mask=True, 
       padding='max_length'
    )
    return tokenized_inputs

# preprocess dataset
tokenized_datasets = dataset.map(group_texts, batched=True, remove_columns=["text"], num_proc=num_proc)
tokenized_datasets.features

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.23.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file vocab.json from cache at /root/.cache/huggingface/hub/mo

The max length for the tokenizer is: 16
                                        text
0                                       text
1                                   tekiture
2                 big horn olive oil company
3                                jacks shack
4                     feeding laramie valley
...                                      ...
999995                            boch honda
999996  allegiance home health and rehab inc
999997                    auto fetish detail
999998         churchfield veterinary centre
999999                planetworks consulting

[1000000 rows x 1 columns]
Dataset({
    features: ['text'],
    num_rows: 1000000
})
     

#0:   0%|          | 0/250 [00:00<?, ?ba/s]

#1:   0%|          | 0/250 [00:00<?, ?ba/s]

#2:   0%|          | 0/250 [00:00<?, ?ba/s]

#3:   0%|          | 0/250 [00:00<?, ?ba/s]

{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'special_tokens_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [ ]:
# from itertools import chain

# # # NOOOOO do not run!!!

# # Main data processing function that will concatenate all texts from our dataset and generate chunks of
# # max_seq_length.
# def group_texts(examples):
#     # Concatenate all texts.
#     concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
#     total_length = len(concatenated_examples[list(examples.keys())[0]])
#     # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
#     # customize this part to your needs.
#     if total_length >= tokenizer.model_max_length:
#         total_length = (total_length // tokenizer.model_max_length) * tokenizer.model_max_length
#     # Split by chunks of max_len.
#     result = {
#         k: [t[i : i + tokenizer.model_max_length] for i in range(0, total_length, tokenizer.model_max_length)]
#         for k, t in concatenated_examples.items()
#     }
#     return result

# tokenized_datasets = tokenized_datasets.map(group_texts, batched=True, num_proc=num_proc)
# # shuffle dataset
# tokenized_datasets = tokenized_datasets.shuffle(seed=34)

# print(f"the dataset contains in total {len(tokenized_datasets)*tokenizer.model_max_length} tokens")

In [15]:
tokenized_datasets = tokenized_datasets.shuffle(seed=34)

In [16]:

# push dataset to hugging face
dataset_id=f"{user_id}/processed_roberta_dataset_padded_med"
tokenized_datasets.push_to_hub(f"{user_id}/processed_roberta_dataset_padded_med")


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
from datasets import load_dataset
import logging
import sys

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
    level=logging.INFO,
)
logger = logging.getLogger(__name__)

In [ ]:
!pip install torch==1.8.1

In [ ]:
import torch
torch.cuda.empty_cache()
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [18]:
from transformers import TrainerCallback, TrainerState, TrainerControl, Trainer, TrainingArguments

class FreezingCallback(TrainerCallback):
    """Callback to gradually unfreeze the model according to a freezing :class:`Schedule` during training. It ensures that the model is always completely unfrozen before saving it to avoid unexpected behaviour."""

    def __init__(self, freezing_schedule, trainer: Trainer, model_config):
        self.model_config = model_config
        self.trainer = trainer
        self.freezing_schedule = freezing_schedule
        self.current_step_idx = 0

    def on_step_begin(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, **kwargs):
        if state.global_step % 1000 == 0:
            self.current_step_idx += 1
            self.freeze_model(self.model_config.num_hidden_layers * 
                              self.freezing_schedule(state.global_step/state.max_steps),
                              self.model_config.num_hidden_layers, int(state.epoch))

    def on_save(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, **kwargs):
        for name, param in self.trainer.model.named_parameters():
            param.requires_grad = True

    def freeze_model(self, freeze_to: int, highest_layer: int, epoch: int):
        print(f"\nEpoch {epoch}: Freezing model to layer {freeze_to} of {highest_layer} layers.")

        for name, param in self.trainer.model.named_parameters():
            for i in range(int(min(freeze_to, highest_layer))):
              if name.startswith(f"roberta.encoder.layer.{i}."):
                  param.requires_grad = False


In [22]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForMaskedLM, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from huggingface_hub import HfFolder
import numpy as np
import math

seed = 34

dataset_id = "ellabettison/processed_roberta_dataset_padded_med"
#tokenizer_id = "ellabettison/finetuned_orgnames_roberta"
tokenizer_id="roberta-base"
hf_hub_token = HfFolder.get_token()
repository_id = "finetuned_orgnames_roberta"
model_config_id = "roberta-base"
mlm_probability = 0.2

# load processed dataset
train_dataset = load_dataset(dataset_id, split="train")
# load trained tokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id, use_auth_token=hf_hub_token)

# load model from config (for training from scratch)
logger.info("Training new model from scratch")
# config = AutoConfig.from_pretrained(model_config_id)
# config.hidden_dropout_prob = 0.1
# model = AutoModelForMaskedLM.from_config(config)

model = AutoModelForMaskedLM.from_pretrained(model_config_id)

logger.info(f"Resizing token embedding to {len(tokenizer)}")
model.resize_token_embeddings(len(tokenizer))

# This one will take care of randomly masking the tokens.
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=mlm_probability, pad_to_multiple_of=8
)

# 12 layers total
# for name, param in model.named_parameters():
#   for i in range(7):
#     if name.startswith(f"roberta.encoder.layer.{i}."):
#       param.requires_grad = False

# for name, param in model.named_parameters():
#   print(name, param.requires_grad)


# hyperparameters
# hyperparameters = {
#     "model_config_id": "roberta-base",
#     "dataset_id": "philschmid/processed_bert_dataset",
#     "tokenizer_id": "ellabettison/finetuned_orgnames_finbert",
#     "repository_id": "roberta-base",
#     "hf_hub_token": HfFolder.get_token(),  # need to be login in with `huggingface-cli login`
#     "max_steps": 10_000,
#     "per_device_train_batch_size": 32,
#     "learning_rate": 5e-5, 
# }

# define our hyperparameters
gaudi_training_args = TrainingArguments(
    output_dir=repository_id,
    # use_habana=True,
    # use_lazy_mode=True,
    per_device_train_batch_size=16,
    learning_rate=2e-5, #5e-5
    seed=seed,
    max_steps=180_000,  #5_000, #_000,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=15_000,
    save_total_limit=20,
    report_to="tensorboard",
    # push to hub parameters
    push_to_hub=True,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
    # pretraining
    ddp_find_unused_parameters=True,
    # throughput_warmup_steps=2,
    #warmup_ratio =0.1,
    lr_scheduler_type="constant", #REMOVE!!
    weight_decay=0.1,
)

# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=gaudi_training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

def sigmoid(x):
  return 1-(1 / (1 + math.exp(-((x*5)-math.e))))

def sigmoid2(x):
  return 1-(0.66 / (1 + math.exp(-((x*6)-math.e))))-0.16

def linear(x):
   return -0.6*x+0.8

#freezing_callback = FreezingCallback(sigmoid2, trainer, model.config)
#trainer.add_callback(freezing_callback)

# train the model
trainer.train()

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.23.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file vocab.json from cache at /root/.cache/huggingface/hub/mo

Step,Training Loss
100,5.236200
200,5.033700
300,4.809200
400,4.938800
500,4.958800
600,4.654400
700,4.697600
800,4.917000
900,4.834300
1000,4.483600


Saving model checkpoint to finetuned_orgnames_roberta/checkpoint-15000
Configuration saved in finetuned_orgnames_roberta/checkpoint-15000/config.json
Model weights saved in finetuned_orgnames_roberta/checkpoint-15000/pytorch_model.bin
tokenizer config file saved in finetuned_orgnames_roberta/checkpoint-15000/tokenizer_config.json
Special tokens file saved in finetuned_orgnames_roberta/checkpoint-15000/special_tokens_map.json
tokenizer config file saved in finetuned_orgnames_roberta/tokenizer_config.json
Special tokens file saved in finetuned_orgnames_roberta/special_tokens_map.json
Saving model checkpoint to finetuned_orgnames_roberta/checkpoint-30000
Configuration saved in finetuned_orgnames_roberta/checkpoint-30000/config.json
Model weights saved in finetuned_orgnames_roberta/checkpoint-30000/pytorch_model.bin
tokenizer config file saved in finetuned_orgnames_roberta/checkpoint-30000/tokenizer_config.json
Special tokens file saved in finetuned_orgnames_roberta/checkpoint-30000/specia

Step,Training Loss
100,5.236200
200,5.033700
300,4.809200
400,4.938800
500,4.958800
600,4.654400
700,4.697600
800,4.917000
900,4.834300
1000,4.483600


Saving model checkpoint to finetuned_orgnames_roberta/checkpoint-180000
Configuration saved in finetuned_orgnames_roberta/checkpoint-180000/config.json
Model weights saved in finetuned_orgnames_roberta/checkpoint-180000/pytorch_model.bin
tokenizer config file saved in finetuned_orgnames_roberta/checkpoint-180000/tokenizer_config.json
Special tokens file saved in finetuned_orgnames_roberta/checkpoint-180000/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=180000, training_loss=4.137714363776313, metrics={'train_runtime': 22145.336, 'train_samples_per_second': 130.05, 'train_steps_per_second': 8.128, 'total_flos': 2.369389169664e+16, 'train_loss': 4.137714363776313, 'epoch': 2.88})

In [23]:
trainer.push_to_hub()

Saving model checkpoint to finetuned_orgnames_roberta
Configuration saved in finetuned_orgnames_roberta/config.json
Model weights saved in finetuned_orgnames_roberta/pytorch_model.bin
tokenizer config file saved in finetuned_orgnames_roberta/tokenizer_config.json
Special tokens file saved in finetuned_orgnames_roberta/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.34k/476M [00:00<?, ?B/s]

Upload file logs/events.out.tfevents.1666101490.9fc6b9835539.76.12:   1%|1         | 3.34k/285k [00:00<?, ?B/s…

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/ellabettison/finetuned_orgnames_roberta
   5eb374c..1bf0843  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/ellabettison/finetuned_orgnames_roberta
   5eb374c..1bf0843  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Masked Language Modeling', 'type': 'fill-mask'}}
To https://huggingface.co/ellabettison/finetuned_orgnames_roberta
   1bf0843..4741fee  main -> main

   1bf0843..4741fee  main -> main



'https://huggingface.co/ellabettison/finetuned_orgnames_roberta/commit/1bf084352d9f2ade11d859a0cda2dd3be13e9f56'

In [ ]:
bert_tokenizer.push_to_hub(tokenizer_id)

In [27]:
import shutil

In [28]:
shutil.move("/content/finetuned_orgnames_roberta.zip", "/content/drive/MyDrive/org_name_parsing/finetuned_orgnames_roberta.zip") 

'/content/drive/MyDrive/org_name_parsing/finetuned_orgnames_roberta.zip'

In [25]:
!zip -r /content/finetuned_orgnames_roberta.zip /content/finetuned_orgnames_roberta

from google.colab import files
files.download("/content/finetuned_orgnames_roberta.zip")

  adding: content/finetuned_orgnames_roberta/ (stored 0%)
  adding: content/finetuned_orgnames_roberta/.gitattributes (deflated 87%)
  adding: content/finetuned_orgnames_roberta/checkpoint-60000/ (stored 0%)
  adding: content/finetuned_orgnames_roberta/checkpoint-60000/merges.txt (deflated 53%)
  adding: content/finetuned_orgnames_roberta/checkpoint-60000/config.json (deflated 50%)
  adding: content/finetuned_orgnames_roberta/checkpoint-60000/tokenizer_config.json (deflated 48%)
  adding: content/finetuned_orgnames_roberta/checkpoint-60000/tokenizer.json (deflated 72%)
  adding: content/finetuned_orgnames_roberta/checkpoint-60000/scheduler.pt (deflated 51%)
  adding: content/finetuned_orgnames_roberta/checkpoint-60000/optimizer.pt (deflated 7%)
  adding: content/finetuned_orgnames_roberta/checkpoint-60000/trainer_state.json (deflated 92%)
  adding: content/finetuned_orgnames_roberta/checkpoint-60000/pytorch_model.bin (deflated 7%)
  adding: content/finetuned_orgnames_roberta/checkpoint

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>